read the data and find the character(choose x[-90 90] y[-125 115] as the final range)

In [ ]:
using CSV
using DataFrames
file = "/home/lingxia/data/seismic_data/BayArea/obspy/stations.csv"
stations = CSV.read(file,DataFrame)
numsta = size(stations,1)
maxx = 0
minn = 100
for i = 1:numsta
    nowi = stations[i,10]
    if nowi > maxx
        maxx = nowi
    elseif nowi < minn
        minn = nowi
    end
end
print(maxx,minn)
# station
# x 85.27638471396759  -85.78970083265496
# y 110.96367321471415 -110.02896866918204

In [ ]:
using CSV
using DataFrames
file = "/home/lingxia/data/seismic_data/BayArea/obspy/events.csv"
events = CSV.read(file,DataFrame)
numsrc = size(events,1)
maxx = 0
minn = 100
for i = 1:numsrc
    nowi = events[i,8]
    if nowi > maxx
        maxx = nowi
    elseif nowi < minn
        minn = nowi
    end
end
print(maxx,minn)
# event
# x 89.08458545263605  -81.93655817886363
# y 110.87770922228904 -121.06632597917981

convert the data to proper type && a small pick to avoid calculate all the data

In [ ]:
multi = 1
allsta = DataFrame(x=[],y=[])
for i = 1 : numsta
    x = stations[i,9]
    y = stations[i,10]
    nx = convert(Int64,round(x*multi))
    ny = convert(Int64,round(y*multi))
    # pick
    if nx > 0 && nx <= 30 && ny >0 && ny <= 40
        push!(allsta.x,nx)
        push!(allsta.y,ny)
    end
end

numsta = size(allsta,1)
print(numsta)

In [ ]:
multi = 1
allsrc = DataFrame(x=[],y=[])
for i = 1 : numsrc
    x = events[i,7]
    y = events[i,8]
    nx = convert(Int64,round(x*multi))
    ny = convert(Int64,round(y*multi))
    if nx > 0 && nx <= 30 && ny >0 && ny <= 40
        push!(allsrc.x,nx)
        push!(allsrc.y,ny)
    end
end

numsrc = size(allsrc,1)
print(numsrc)

velocity model

In [15]:
m = 30
n = 40
h = 0.1
len = 5 # the length of the grid

f = ones(n, m)
for i = 0 : n-1
    for j = 0 : m-1
        ii = (i - i%len)/len
        jj = (j - j%len)/len
        if ( ii + jj )%2 == 0
            f[i+1,j+1] = 2
        end
    end
end

use the test.jl

In [ ]:
push!(LOAD_PATH,"../src/")
using ADCME
using ADEikonal
using PyCall
using LinearAlgebra
using PyPlot
using CSV
using DataFrames
using Random
Random.seed!(233)

reset_default_graph()

u = PyObject[]
for i=1:numsrc
    push!(u,eikonal(f,allsrc[i,1],allsrc[i,2],h))
end

sess = Session()
uobs = run(sess, u)

F = Variable(ones(n, m))
u = PyObject[]
for i=1:numsrc
    push!(u,eikonal(F,allsrc[i,1],allsrc[i,2],h))
end

loss = sum([sum((uobs[i][allsta[j,2],allsta[j,1]] - u[i][allsta[j,2],allsta[j,1]])^2) for i = 1:numsrc for j=1:numsta])

init(sess)
@show run(sess, loss)

BFGS!(sess, loss, 400,var_to_bounds=Dict(F=>(0.5,100.0)))

figure(figsize=(10, 4))
subplot(121)
pcolormesh(f)
colorbar()
title("True")
scatter(allsrc[:,1],allsrc[:,2],label="event")
scatter(allsta[:,1],allsta[:,2],label="station")
legend()
subplot(122)
pcolormesh(run(sess,F),vmin=0.8,vmax=2.2)#vmin&vmax
colorbar()
title("Inverted")
savefig("/home/lingxia/ADEikonal.jl/tests/2D_results/real_2.png")